In [74]:
#impor packages
import pandas as pd
import numpy as np
import itertools
import os
import pickle
import json
import re

In [54]:
input_path = "../datasets/raw/"

In [55]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'OUTPUTEVENTS.csv', dtype ="object")

In [56]:
# read D_LABITEMS.csv file
df_D_ITEMS = pd.read_csv(filepath_or_buffer=input_path+'D_ITEMS.csv', dtype = "object")

In [57]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4349218 entries, 0 to 4349217
Data columns (total 13 columns):
ROW_ID        4349218 non-null object
SUBJECT_ID    4349218 non-null object
HADM_ID       4344379 non-null object
ICUSTAY_ID    4342028 non-null object
CHARTTIME     4349218 non-null object
ITEMID        4349218 non-null object
VALUE         4260669 non-null object
VALUEUOM      4312967 non-null object
STORETIME     4349218 non-null object
CGID          4349218 non-null object
STOPPED       0 non-null object
NEWBOTTLE     0 non-null object
ISERROR       0 non-null object
dtypes: object(13)
memory usage: 431.4+ MB


In [58]:
df_D_ITEMS.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12487 entries, 0 to 12486
Data columns (total 10 columns):
ROW_ID          12487 non-null object
ITEMID          12487 non-null object
LABEL           12483 non-null object
ABBREVIATION    2992 non-null object
DBSOURCE        12487 non-null object
LINKSTO         12487 non-null object
CATEGORY        6049 non-null object
UNITNAME        1234 non-null object
PARAM_TYPE      2992 non-null object
CONCEPTID       0 non-null object
dtypes: object(10)
memory usage: 975.6+ KB


In [59]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 1)

In [60]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,21219,177991,225765,2142-09-08 10:00:00,40055,200,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,21219,177991,225765,2142-09-08 12:00:00,40055,200,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,21219,177991,225765,2142-09-08 13:00:00,40055,120,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,21219,177991,225765,2142-09-08 14:00:00,40055,100,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,21219,177991,225765,2142-09-08 16:00:00,40055,200,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN


In [61]:
df_D_ITEMS.head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [62]:
# map ITMEID labels from D_ITEMID
df = df.merge(df_D_ITEMS[['ITEMID','LABEL']], how='left',left_on='ITEMID', right_on='ITEMID')

In [63]:
df = df.drop(labels=['CHARTTIME','CGID','STOPPED','NEWBOTTLE','ISERROR'], axis = 'columns')

In [64]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,VALUE,VALUEUOM,STORETIME,LABEL
0,21219,177991,225765,40055,200,ml,2142-09-08 12:08:00,Urine Out Foley
1,21219,177991,225765,40055,200,ml,2142-09-08 12:08:00,Urine Out Foley
2,21219,177991,225765,40055,120,ml,2142-09-08 13:39:00,Urine Out Foley
3,21219,177991,225765,40055,100,ml,2142-09-08 16:17:00,Urine Out Foley
4,21219,177991,225765,40055,200,ml,2142-09-08 16:17:00,Urine Out Foley


In [66]:
# replace missing values with 'NaN'
df = df.fillna('NaN')

In [67]:
# Merge ITEMID with respective VALUE and UOM
df['VALUE_UOM'] = df['ITEMID']+'-'+df['VALUE']+'-'+df['VALUEUOM']

In [70]:
# replace VALUE_UOM with 'NaN' where VALUE is missing
df.loc[df['VALUE']=='NaN','VALUE_UOM'] = 'NaN'

In [75]:
# remove special charecters from LABELS
df['LABEL'] = df['LABEL'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

In [76]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,VALUE,VALUEUOM,STORETIME,LABEL,VALUE_UOM
0,21219,177991,225765,40055,200,ml,2142-09-08 12:08:00,Urine Out Foley,40055-200-ml
1,21219,177991,225765,40055,200,ml,2142-09-08 12:08:00,Urine Out Foley,40055-200-ml
2,21219,177991,225765,40055,120,ml,2142-09-08 13:39:00,Urine Out Foley,40055-120-ml
3,21219,177991,225765,40055,100,ml,2142-09-08 16:17:00,Urine Out Foley,40055-100-ml
4,21219,177991,225765,40055,200,ml,2142-09-08 16:17:00,Urine Out Foley,40055-200-ml


In [77]:
# Create outputevents
df['outputevents'] = df['LABEL']+' '+df['VALUE_UOM']

In [78]:
# drop redundant columns
df = df.drop(labels = ['ITEMID','VALUE','VALUEUOM','LABEL','VALUE_UOM'], axis = 'columns')

In [89]:
# split outputevents on whitespace
df['outputevents'] = df['outputevents'].map(lambda x: x.split()) 

In [91]:
df.head()

,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STORETIME,outputevents
0,21219,177991,225765,2142-09-08 12:08:00,"[Urine, Out, Foley, 40055-200-ml]"
1,21219,177991,225765,2142-09-08 12:08:00,"[Urine, Out, Foley, 40055-200-ml]"
2,21219,177991,225765,2142-09-08 13:39:00,"[Urine, Out, Foley, 40055-120-ml]"
3,21219,177991,225765,2142-09-08 16:17:00,"[Urine, Out, Foley, 40055-100-ml]"
4,21219,177991,225765,2142-09-08 16:17:00,"[Urine, Out, Foley, 40055-200-ml]"


In [92]:
# create output path
mypath_output = "../datasets/outputevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [94]:
# export the dataframe to JSON format
df.to_json(mypath_output+"outputevents.json",orient = 'records') 